# **Hash Sharding**

This notebook:

1. shows hash tables
2. shows plan when joining with other tables

# Query 1

Creates 2 table distributing them on the same key

In [ ]:
-- Query 1
create table hashtable 
(
	id int 
)
with
(
	distribution = hash(id)
);


create table hashtable2
(
	id int 
)
with
(
	distribution = hash(id)
);

declare @i as int = 1
while (@i <= 25)
begin
	insert into hashtable (id) values(@i);
	set @i += 1;

end


while (@i <= 25)
begin
	insert into hashtable2 (id) values(@i);
	set @i += 1;

end

DBCC PDW_SHOWSPACEUSED('dbo.hashtable')

# Query 2

Shows plan when joining distributed tabl with the same key

In [ ]:
-- Query 2
SELECT top(100000) * 
from dbo.hashtable as t 
    inner join dbo.hashtable2 as t2 on t.id = t2.id
option (label = 'SELECT FROM hash JOIN hash');

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'SELECT FROM hash JOIN hash' 
    order by start_time desc 
)

# Query 3

Shows plan when joining on different distribution keys

In [ ]:
-- Query 3
select top(100000) * 
from dbo.hashtable as t 
    inner join dbo.hashtable2 as t2 on t.id = t2.id * 2
option (label = 'SELECT FROM hash JOIN hash another distro');

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'SELECT FROM hash JOIN hash another distro' 
    order by start_time desc 
)

# Query 4

Show plan when joining replicate and hash table

In [ ]:
-- Query 4
select * 
from dbo.hashtable as t 
    inner join dbo.Geography_replicated as g on t.id = g.GeographyID
option (label = 'SELECT FROM hash JOIN replicated');

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'SELECT FROM hash JOIN replicated' 
    order by start_time desc 
)